In [2]:
import json
import os
import copy
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import sys
sys.path.append("../")

from cfmining.algorithms import MAPOFCEM
from cfmining.predictors import GeneralClassifier_Shap
from cfmining.action_set import ActionSet
from cfmining.utils import get_data_model, DeepPipeExplainer
from cfmining.baselines import Bruteforce, MAPOCAM, Nice, Dice


%load_ext autoreload
%autoreload 2

In [3]:
SEED = 0

## Helper

In [4]:
def run_experiments(
        method,
        individuals, 
        model, 
        output_file = None,
    ):
    results = []

    if not output_file is None:
        folder = "/".join(output_file.split("/")[:-1])
        if not os.path.exists(folder):
            os.makedirs(folder, exist_ok = True)

    for i in tqdm(range(len(individuals))):
        individual = individuals.iloc[i]
        try:
            model.clear_cache()
        except:
            pass
        start = time.time()
        method.fit(individual.values)
        end = time.time()

        solutions = method.solutions
        
        results.append({
            "individual" : individual.values.tolist(),
            "prob" : model.predict_proba(individual.values),
            "time" : end - start,
            "n_solutions" : len(method.solutions),
            "solutions" : solutions,
        })

        if output_file is not None:
            pd.DataFrame(results).to_csv(output_file, index=False)

        

    results = pd.DataFrame(results)
    if output_file is not None:
        results.to_csv(output_file, index=False)
    else:
        return results

## German

In [5]:
X_train, Y_train, model, outlier_detection, individuals = get_data_model("german", "MLPClassifier")
individuals = individuals.sample(n = 50, random_state=SEED)

In [6]:
not_mutable_features = ['Age', 'OwnsHouse', 'isMale', 'JobClassIsSkilled', 'Single', 'ForeignWorker', 'RentsHouse']
mutable_features = [feat for feat in X_train.columns if feat not in not_mutable_features]
continuous_features = ['Age', 'LoanDuration', 'LoanAmount', 'LoanRateAsPercentOfIncome','YearsAtCurrentHome']
categoric_features = [col for col in X_train.columns if col not in continuous_features]

In [7]:
action_set = ActionSet(X = X_train, default_step_size = 0.05, mutable_features = mutable_features)
deep_pipe_explainer = DeepPipeExplainer(model, X_train.sample(100))
model_shap = GeneralClassifier_Shap(model, outlier_detection, X_train, shap_explainer="custom", threshold = 0.5, explainer = deep_pipe_explainer)
model_shap_simple = GeneralClassifier_Shap(model, outlier_detection, X_train)

### MAPOFCEM

In [8]:
method = MAPOFCEM(
    action_set = action_set,
    classifier = model_shap,
    compare = "percentile",
    max_changes = 3
)

run_experiments(
    method,
    individuals=individuals,
    model=model_shap,
    output_file=f"../results/mlp/german/mapofcem_percentile.csv"
)

  0%|                                                                                                                                   | 0/50 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:52<00:00,  1.05s/it]


In [9]:
method = MAPOFCEM(
    action_set = action_set,
    classifier = model_shap_simple,
    compare = "percentile",
    max_changes = 3
)

run_experiments(
    method,
    individuals=individuals,
    model=model_shap,
    output_file=f"../results/mlp/german/mapofcem_perm_percentile.csv"
)

  0%|                                                                                                                                   | 0/50 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [02:46<00:00,  3.33s/it]


### MAPOCAM

In [11]:
model_shap.use_predict_max = False # to not use max prediction
# little fix to action set
action_set_ = copy.deepcopy(action_set)
for feat in action_set_:
    feat.flip_direction = 1
    feat.update_grid()

method = MAPOCAM(
    action_set_,
    model_shap,
    criteria = "percentile",
    max_changes=3
)

run_experiments(
    method,
    individuals=individuals,
    model=model_shap,
    output_file=f"../results/mlp/german/mapocam_percentile.csv"
)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [01:14<00:00,  1.49s/it]


### Bruteforce

In [14]:
model_shap.use_predict_max = False # to not use prediction max
# little fix to action set
action_set_ = copy.deepcopy(action_set)
for feat in action_set_:
    feat.flip_direction = 1
    feat.update_grid()


method = Bruteforce(
    action_set_,
    model_shap,
    criteria = "percentile",
    max_changes=3
)

run_experiments(
    method,
    individuals=individuals,
    model=model_shap,
    output_file=f"../results/mlp/german/bruteforce_percentile.csv"
)

  6%|███████▍                                                                                                                   | 3/50 [02:57<46:22, 59.20s/it]


KeyboardInterrupt: 

### DICE

In [12]:
method = Dice(
    X_train,
    Y_train,
    model_shap.clf,
    n_cfs = 1,
    mutable_features = mutable_features,
)

run_experiments(
    method,
    individuals = individuals,
    model = model_shap,
    output_file=f"../results/mlp/german/dice_1sol.csv"
)

  0%|                                                                                                                                   | 0/50 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:08<00:00,  5.66it/s]


### NICE

In [13]:
method = Nice(
    X_train,
    Y_train,
    model = model,
    cat_features = categoric_features,
)

run_experiments(
    method,
    individuals = individuals,
    model = model_shap,
    output_file=f"../results/mlp/german/nice.csv"
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


  0%|                                                                                                                                   | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was

## Taiwan

In [15]:
X_train, Y_train, model, outlier_detection, individuals = get_data_model("taiwan", "MLPClassifier")
#convert bools to int
X_train = X_train.astype(int)
individuals = individuals.astype(int)
individuals = individuals.sample(n = 50, random_state=SEED)

In [16]:
not_mutable_features = ['Single', 'Age_in_25_to_40', 'Married', 'Age_lt_25', 'Age_in_40_to_59', 'Age_geq_60', 'EducationLevel']
mutable_features = [feat for feat in X_train.columns if feat not in not_mutable_features]
continuous_features = ["LIMIT_BAL", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3",
                       "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1",
                       "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5",
                       "PAY_AMT6", "MaxBillAmountOverLast6Months", "MaxPaymentAmountOverLast6Months",
                       "MostRecentBillAmount", "MostRecentPaymentAmount", "MostRecentPaymentAmount", "TotalMonthsOverdue"]
categoric_features = [col for col in X_train.columns if col not in continuous_features]

In [17]:
action_set = ActionSet(X = X_train, default_step_size = 0.1, mutable_features = mutable_features)
deep_pipe_explainer = DeepPipeExplainer(model, X_train.sample(100))
model_shap = GeneralClassifier_Shap(model, outlier_detection, X_train, shap_explainer="custom", threshold = 0.5, explainer = deep_pipe_explainer)
model_shap_simple = GeneralClassifier_Shap(model, outlier_detection, X_train)

PermutationExplainer explainer: 1001it [00:29, 23.59it/s]                                                                                                      


### MAPOFCEM

In [18]:
method = MAPOFCEM(
    action_set = action_set,
    classifier = model_shap_simple,
    compare = "percentile",
    max_changes = 3,
    outlier_percentile = 0.01
)

run_experiments(
    method,
    individuals=individuals.iloc[1:],
    model=model_shap,
    output_file=f"../results/mlp/taiwan/mapofcem_percentile.csv"
)

  0%|                                                                                                                                   | 0/49 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [19:33<00:00, 23.96s/it]


In [19]:
method = MAPOFCEM(
    action_set = action_set,
    classifier = model_shap_simple,
    compare = "percentile",
    max_changes = 3,
    outlier_percentile=0.01
)

run_experiments(
    method,
    individuals=individuals,
    model=model_shap,
    output_file=f"../results/mlp/taiwan/mapofcem_perm_percentile.csv"
)

  0%|                                                                                                                                   | 0/50 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [20:19<00:00, 24.39s/it]


### MAPOCAM

In [20]:
model_shap.use_predict_max = False # to not use max prediction
# little fix to action set
action_set_ = copy.deepcopy(action_set)
for feat in action_set_:
    feat.flip_direction = 1
    feat.update_grid()

method = MAPOCAM(
    action_set_,
    model_shap,
    criteria = "percentile",
    max_changes=3
)

run_experiments(
    method,
    individuals=individuals,
    model=model_shap,
    output_file=f"../results/mlp/taiwan/mapocam_percentile.csv"
)

  0%|                                                                                                                                   | 0/50 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [26:51<00:00, 32.22s/it]


### DICE

In [21]:
method = Dice(
    X_train,
    Y_train,
    model_shap.clf,
    n_cfs = 1,
    mutable_features = mutable_features,
)

run_experiments(
    method,
    individuals = individuals,
    model = model_shap,
    output_file=f"../results/mlp/taiwan/dice_1sol.csv"
)

  0%|                                                                                                                                   | 0/50 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:13<00:00,  3.72it/s]


### NICE

In [22]:
method = Nice(
    X_train,
    Y_train,
    model = model,
    cat_features = categoric_features,
)

run_experiments(
    method,
    individuals = individuals,
    model = model_shap,
    output_file=f"../results/mlp/taiwan/nice.csv"
)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
  0%|                                                                                                                                   | 0/50 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but 